In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from tqdm import tqdm
from copy import deepcopy
import torch.nn.functional as F






class PassportHook:
    """Hook that applies passport transformation: alpha * output + beta"""

    def __init__(self, conv_layer, passport_activation):

        self.conv_layer = conv_layer
        self.passport_activation = passport_activation.clone()
        self.hook = None

        # Ensure passport is 4D [1, C, H, W]
        if self.passport_activation.dim() == 2:
            self.passport_activation = self.passport_activation.unsqueeze(0)

    def compute_alpha_beta(self, output_shape=None):
        # passport_activation shape: [1, seq_len, hidden_size]
        # We treat it as [1, hidden_size, seq_len, 1] to use conv2d
        pa = self.passport_activation  # [1, seq_len, hidden]
        # print(pa.shape)
        # Normalize to [1, seq_len, hidden]
        while pa.dim() > 3:
            pa = pa.squeeze(0)
        if pa.dim() == 2:
            pa = pa.unsqueeze(0)  # [1, seq_len, hidden]
        # Average over seq_len -> [1, hidden]
        pa_mean = pa.mean(dim=(0,1))  # [hidden]
        w = self.conv_layer.weight  # [out_features, in_features]
        # print(w.shape)
        # print((pa_mean.T).shape)
        # print(self.conv_layer.weight.shape)
        # print(pa_mean.unsqueeze(-1).shape)



        alpha = w @ pa_mean.unsqueeze(-1)  # [out_features, 1]
        beta = alpha.clone()

        return alpha, beta

    def hook_fn(self, module, input, output):
        """Apply passport transformation: alpha * output + beta"""
        alpha, beta = self.compute_alpha_beta()
        alpha = alpha.to(output.device) #[out, 1]
        beta = beta.to(output.device)

        # output shape: [batch, seq_len, hidden] or [batch, hidden]
        if output.dim() == 3:
            # [batch, seq, hidden] -> scale along hidden dim
            transformed_output = alpha.squeeze(-1) * output + beta.squeeze(-1)
        else:
            transformed_output = alpha.squeeze(-1) * output + beta.squeeze(-1)


        return transformed_output

    def register(self, module):
        """Register the hook to the module"""
        self.hook = module.register_forward_hook(self.hook_fn)

    def remove(self):
        """Remove the hook"""
        if self.hook:
            self.hook.remove()





class PassportSentiment:
    def __init__(self, model_id="prajjwal1/bert-tiny", device="cuda"):
        self.device = device
        self.model_id = model_id
        
        # Chargement du modele
        # self.model = AutoModelForSequenceClassification.from_pretrained(
        #     model_id, num_labels=2, use_safetensors=True
        # ).to(device)
        # self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        config = BertConfig.from_pretrained(model_id, num_labels=2)
        self.model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
        self.tokenizer=BertTokenizer.from_pretrained(model_id)
        # Configuration par defaut
        self.config = {
            "layer_name": "bert.encoder.layer.1.output.dense",  # Derniere couche encoder
             # "layer_name": "classifier",
            "watermark_len": 64,
            "lr": 5e-5,
            "lambda_wat": 1.0,
            "epochs": 30,
            "gamma": 0.1,

        }
        
        self.saved_keys = {}
        
    def list_layers(self):
        """Affiche les couches disponibles pour le watermarking."""
        print("Couches disponibles (modules):")
        for name, module in self.model.named_modules():
            if name:
                print(f"  {name}")

    def _get_layer(self, model, layer_name):
        """Navigate to a layer by dotted path."""
        layer = model
        for part in layer_name.split('.'):
            if part.isdigit():
                layer = layer[int(part)]
            else:
                layer = getattr(layer, part)
        return layer

    def _get_target_layer(self, model, layer_name):
        """Recupere le module PyTorch correspondant au nom."""
        for name, module in model.named_modules():
            if name == layer_name:
                return module
        raise ValueError(f"Couche {layer_name} introuvable.")
    
    def _generate_passport_activation(self, model):
        """
        Generate passport activation using a fixed dummy text input,
        capturing the output of the target layer.
        """
        # Fixed trigger: dummy token IDs (e.g., all [MASK] tokens)
        trigger_input_ids = torch.ones(1, 32, dtype=torch.long).to(self.device)  # [1, seq_len]
        trigger_attention_mask = torch.ones(1, 32, dtype=torch.long).to(self.device)

        target_layer = self._get_layer(model, self.config["layer_name"])



        # Hook to capture activation
        activation = {}
        def capture_hook(module, input, output):
            if isinstance(input, tuple):
                activation['output'] = input[0].clone()
            else:
                activation['output'] = input.clone()


        hook = target_layer.register_forward_hook(capture_hook)

        # Forward pass
        with torch.no_grad():
                  _ = model(input_ids=trigger_input_ids, attention_mask=trigger_attention_mask)


        hook.remove()
        trigger_input = (trigger_input_ids, trigger_attention_mask)

        return activation['output'], (trigger_input_ids, trigger_attention_mask)





    @staticmethod
    def _sign_loss(watermark, alpha, gamma=0.1):
        """
        Compute sign loss R(P, B) = sum_i max(gamma - alpha_i * b_i, 0)
        Forces alpha signs to match watermark.
        """
        if alpha.dim() > 1:
            alpha = alpha.view(-1)
        watermark = watermark.view(-1)

        if not torch.is_tensor(gamma):
            gamma = torch.tensor(gamma, dtype=alpha.dtype, device=alpha.device)

        values = gamma - alpha * watermark
        R = torch.sum(torch.clamp(values, min=0))
        return R

    @staticmethod
    def _check_sign_match(alpha, watermark):
        """Check how many signs match between alpha and watermark"""
        if alpha.dim() > 1:
            alpha = alpha.view(-1)
        alpha_signs = torch.sign(alpha)
        matches = (alpha_signs == watermark).float()
        accuracy = torch.mean(matches).item()
        return accuracy, alpha_signs
    
    def embed(self, train_dataloader, eval_dataloader=None):

        print(f"--- Demarrage Embedding Passport ({self.config['layer_name']}) ---")
        

        watermarked_model = self.model
        watermarked_model.train()

        # 1. Generate passport activation from fixed trigger input
        passport_activation, trigger_input = self._generate_passport_activation(watermarked_model)


        print(f"Passport activation shape: {passport_activation.shape}")
        # 2. Get target layer and create passport hook


        target_layer = self._get_layer(watermarked_model, self.config["layer_name"])
        passport_hook = PassportHook(target_layer, passport_activation)


        # 3. Generate watermark matching output dimension of target layer
        out_features = target_layer.weight.shape[0]
        watermark = torch.randint(0, 2, (out_features,), dtype=torch.float32).to(self.device) * 2 - 1


        print(f"Watermark size: {len(watermark)} | First 10: {watermark[:10].tolist()}")


        optimizer = torch.optim.AdamW(watermarked_model.parameters(), lr=self.config["lr"])
        mse_loss = nn.MSELoss()

        gamma = self.config["gamma"]
        
        ce_loss = nn.CrossEntropyLoss()
        bce_loss = nn.BCELoss()
        
        # 5. Boucle d'entrainement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            correct = 0
            total = 0
            
            for batch in pbar:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                
                # --- A. Entrainement ProjectionNet ---
                optimizer.zero_grad()
                

                # Tache principale (classification)
                outputs = watermarked_model(input_ids=input_ids, attention_mask=attention_mask)
                l_main = ce_loss(outputs.logits, labels)
                
                alpha, beta = passport_hook.compute_alpha_beta()
                l_sign = self._sign_loss(watermark, alpha, gamma)

                # Loss totale
                l_total = l_main + self.config["lambda_wat"] * l_sign
                
                l_total.backward()
                optimizer.step()
                
                # Metrics
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                sign_acc, alpha_signs = self._check_sign_match(alpha, watermark)
                ber = 1.0 - sign_acc
                acc = 100. * correct / total
                pbar.set_postfix(L_Main=f"{l_main.item():.3f}", L_Wat=f"{l_sign.item():.3f}",
                                BER=f"{ber:.2f}", Acc=f"{acc:.1f}%")
            
            # Evaluation en fin d'epoch
            if eval_dataloader:
                eval_acc = self.evaluate(watermarked_model, eval_dataloader)
                print(f"Epoch {epoch+1} | Eval Accuracy: {eval_acc:.2f}%")
        
        # Sauvegarde des cles
        self.saved_keys = {
            "watermark": watermark,
            "watermarked_model": watermarked_model,
            "passport_activation": passport_activation,
            "passport_hook": passport_hook,
            "trigger_input": trigger_input,

        }
        
        final_ber, _ = self.extract(watermarked_model)
        print(f"\n--- Embedding termine | BER final: {final_ber:.2f} ---")
        return watermarked_model
    
    def evaluate(self, model, dataloader):
        """Evalue la precision du modele."""
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        model.train()
        return 100. * correct / total
    
    def extract(self, suspect_model=None):
        """
        Extrait la marque d'un modele suspect.
        """
        if suspect_model is None:
            suspect_model = self.saved_keys["watermarked_model"]

        watermark = self.saved_keys["watermark"]
        passport_activation = self.saved_keys["passport_activation"]
        
        suspect_model.eval()
        # Get target layer from suspect model
        target_layer = self._get_layer(suspect_model, self.config["layer_name"])

        # Create new hook with same passport activation
        suspect_hook = PassportHook(target_layer, passport_activation)

        # Compute alpha
        with torch.no_grad():
            alpha, beta = suspect_hook.compute_alpha_beta()
            sign_acc, alpha_signs = self._check_sign_match(alpha, watermark)
            ber = 1.0 - sign_acc
        

        
        print(f"BER Extrait : {ber:.2f}")
        return ber, alpha_signs
    
    @staticmethod
    def _compute_ber(alpha_signs, watermark):
        return ((alpha_signs != watermark).float()).mean().item()

print("Classes chargees (PassportSentiment, FeatureHook, ProjectionNet)")

/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classes chargees (PassportSentiment, FeatureHook, ProjectionNet)


In [2]:
# --- Chargement des donnees SST-2 ---

# Charger le dataset SST-2 (sentiment binaire)
dataset = load_dataset("glue", "sst2")

# Initialiser le watermarker
passport = PassportSentiment(device="cuda" if torch.cuda.is_available() else "cpu")

# Afficher les couches disponibles
passport.list_layers()

# Tokenization
def tokenize_function(examples):
    return passport.tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# DataLoaders
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=64, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=64)

print(f"\nTrain samples: {len(tokenized_dataset['train'])}")
print(f"Eval samples: {len(tokenized_dataset['validation'])}")

Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1676.05it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.LayerNor

Couches disponibles (modules):
  bert
  bert.embeddings
  bert.embeddings.word_embeddings
  bert.embeddings.position_embeddings
  bert.embeddings.token_type_embeddings
  bert.embeddings.LayerNorm
  bert.embeddings.dropout
  bert.encoder
  bert.encoder.layer
  bert.encoder.layer.0
  bert.encoder.layer.0.attention
  bert.encoder.layer.0.attention.self
  bert.encoder.layer.0.attention.self.query
  bert.encoder.layer.0.attention.self.key
  bert.encoder.layer.0.attention.self.value
  bert.encoder.layer.0.attention.self.dropout
  bert.encoder.layer.0.attention.output
  bert.encoder.layer.0.attention.output.dense
  bert.encoder.layer.0.attention.output.LayerNorm
  bert.encoder.layer.0.attention.output.dropout
  bert.encoder.layer.0.intermediate
  bert.encoder.layer.0.intermediate.dense
  bert.encoder.layer.0.intermediate.intermediate_act_fn
  bert.encoder.layer.0.output
  bert.encoder.layer.0.output.dense
  bert.encoder.layer.0.output.LayerNorm
  bert.encoder.layer.0.output.dropout
  bert.enc

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 33429.45 examples/s]


Train samples: 67349
Eval samples: 872


In [3]:

# Lancer l'embedding
watermarked_model = passport.embed(
    train_dataloader, 
    eval_dataloader
)

--- Demarrage Embedding Passport (bert.encoder.layer.1.output.dense) ---
Passport activation shape: torch.Size([1, 32, 512])
Watermark size: 128 | First 10: [-1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0]


Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 215.36it/s, Acc=79.8%, BER=0.00, L_Main=0.299, L_Wat=0.000]


Epoch 1 | Eval Accuracy: 81.08%


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 210.75it/s, Acc=88.5%, BER=0.00, L_Main=0.298, L_Wat=0.000]


Epoch 2 | Eval Accuracy: 82.11%


...

Epoch 28 | Eval Accuracy: 79.24%


Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 240.25it/s, Acc=98.7%, BER=0.00, L_Main=0.010, L_Wat=0.000]


Epoch 29 | Eval Accuracy: 78.21%


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 240.65it/s, Acc=98.7%, BER=0.00, L_Main=0.387, L_Wat=0.000]


Epoch 30 | Eval Accuracy: 79.01%
BER Extrait : 0.00

--- Embedding termine | BER final: 0.00 ---


In [4]:
# --- Test d'Extraction ---

print("=" * 50)
print("TEST D'EXTRACTION")
print("=" * 50)

# Extraction sur le modele watermarke
print("\n1. Extraction sur le modele watermarke:")
ber_watermarked, _ = passport.extract(watermarked_model)

# Extraction sur un modele vierge (non-watermarke)
print("\n2. Extraction sur un modele vierge (devrait avoir BER ~0.5):")
model_id="prajjwal1/bert-tiny"
config = BertConfig.from_pretrained(model_id, num_labels=2)
fresh_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device=passport.device)

ber_fresh, _ = passport.extract(fresh_model)

print(f"\nResume:")
print(f"  - BER modele watermarke: {ber_watermarked:.2f} (attendu: ~0.0)")
print(f"  - BER modele vierge: {ber_fresh:.2f} (attendu: ~0.5)")

TEST D'EXTRACTION

1. Extraction sur le modele watermarke:
BER Extrait : 0.00

2. Extraction sur un modele vierge (devrait avoir BER ~0.5):


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 3057.87it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.LayerNor

BER Extrait : 0.52

Resume:
  - BER modele watermarke: 0.00 (attendu: ~0.0)
  - BER modele vierge: 0.52 (attendu: ~0.5)


In [7]:
# --- Classes Utilitaires ---

class FeatureHook:
    """Intercepte les activations d'une couche specifique."""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module, input, output):
        # Pour les transformers, output peut etre un tuple
        if isinstance(output, tuple):
            self.features = output[0]
        else:
            self.features = output

    def close(self):
        self.hook.remove()


def run_distillation_attack(passport_obj, train_dataloader, eval_dataloader, epochs=5, lr=5e-5):
    """
    Tente de transferer la fonctionnalite du modele watermarke vers un modele vierge.
    Verifie si la marque (basee sur les activations) survit.
    """
    device = passport_obj.device
    
    # 1. Teacher (Gele)
    teacher_model = passport_obj.saved_keys["watermarked_model"]
    teacher_model.eval()
    for p in teacher_model.parameters():
        p.requires_grad = False
    
    # 2. Student (Vierge - Meme architecture)
    config = BertConfig.from_pretrained(model_id, num_labels=2)
    print("\n--- Initialisation du Student ---")
    student_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
    student_model.train()
    
    # Verification initiale
    print("\n[Check] BER Teacher:")
    teacher_ber, _ = passport_obj.extract(teacher_model)
    print("[Check] BER Student (Avant):")
    student_ber, _ = passport_obj.extract(student_model)
    
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=lr)
    history = {"loss": [], "ber": [], "acc": []}
    
    print(f"\n--- Distillation ({epochs} epochs) ---")
    feature_criterion = nn.MSELoss()
    for epoch in range(epochs):
        pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0
        
        student_model.train()
        
        for batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            optimizer.zero_grad()
            teacher_layer = passport._get_layer(teacher_model, passport.config["layer_name"])
            hook = FeatureHook(teacher_layer)
            # Output Matching (Soft Labels)
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits
                teacher_probs = F.softmax(teacher_logits / 2.0, dim=-1)
                t_features = hook.features
            hook.close()

            student_layer = passport._get_layer(student_model, passport.config["layer_name"])
            hook = FeatureHook(student_layer)
            student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits
            student_log_probs = F.log_softmax(student_logits / 2.0, dim=-1)
            s_features = hook.features
            hook.close()
            # print(f"Teacher features shape: {t_features.shape} | Student features shape: {s_features.shape}")
            # t_mean, t_var = compute_stats(t_features)
            # s_mean, s_var = compute_stats(s_features)
            loss_feat = feature_criterion(s_features.mean(dim=[0, 1]), t_features.mean(dim=[0, 1])) + feature_criterion(s_features.var(dim=[0, 1]), t_features.var(dim=[0, 1]))


            # KL Divergence Loss
            loss_kl = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (2.0 ** 2)
            loss= loss_kl +loss_feat
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix(Loss=f"{loss.item():.4f}")
        
        # Evaluation
        avg_loss = running_loss / len(train_dataloader)
        history["loss"].append(avg_loss)
        
        current_ber, pred_wm_prob= passport_obj.extract(student_model)
        history["ber"].append(current_ber)
        
        eval_acc = passport_obj.evaluate(student_model, eval_dataloader)
        history["acc"].append(eval_acc)

        # Convert {-1, +1} to {0, 1} for BCE
        pred_binary = (pred_wm_prob + 1) / 2      # {-1,+1} -> {0,1}
        wm_binary = (passport_obj.saved_keys["watermark"] + 1) / 2  # {-1,+1} -> {0,1}

        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | BER: {current_ber:.2f} | Acc: {eval_acc:.1f}% | err_wat: {nn.BCELoss()(pred_binary, wm_binary).item():.4f}")
        

    
    return student_model, history

In [8]:
# --- Lancer l'attaque par distillation ---

student_model, attack_history = run_distillation_attack(
    passport,
    train_dataloader, 
    eval_dataloader,
    epochs=1000
)


--- Initialisation du Student ---


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 3340.98it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.LayerNor


[Check] BER Teacher:
BER Extrait : 0.00
[Check] BER Student (Avant):
BER Extrait : 0.52

--- Distillation (1000 epochs) ---


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 188.87it/s, Loss=1.1069]


BER Extrait : 0.36
Epoch 1 | Loss: 1.4262 | BER: 0.36 | Acc: 80.7% | err_wat: 35.9375


...

BER Extrait : 0.25
Epoch 654 | Loss: 0.0059 | BER: 0.25 | Acc: 78.3% | err_wat: 25.0000


Epoch 655: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:06<00:00, 173.90it/s, Loss=0.0053]


BER Extrait : 0.26
Epoch 655 | Loss: 0.0052 | BER: 0.26 | Acc: 77.6% | err_wat: 25.7812


Epoch 656:  69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                           | 726/1053 [00:04<00:01, 177.46it/s, Loss=0.0039]


KeyboardInterrupt: 

In [ ]:
# --- Visualisation des Resultats ---

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(attack_history["loss"], 'b-o')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Distillation Loss")
axes[0].set_title("Loss durant Distillation")
axes[0].grid(True)

# BER
axes[1].plot(attack_history["ber"], 'r-o')
axes[1].axhline(y=0.5, color='gray', linestyle='--', label='Random (0.5)')
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("BER")
axes[1].set_title("BER du Student (Watermark)")
axes[1].set_ylim([0, 1])
axes[1].legend()
axes[1].grid(True)

# Accuracy
axes[2].plot(attack_history["acc"], 'g-o')
axes[2].set_xlabel("Epoch")
axes[2].set_ylabel("Accuracy (%)")
axes[2].set_title("Accuracy du Student")
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Resume final
print("\n" + "=" * 50)
print("RESUME FINAL")
print("=" * 50)
teacher_ber, _ = passport.extract(passport.saved_keys['watermarked_model'])
print(f"BER Teacher (watermarke): {teacher_ber:.2f}")
print(f"BER Student (apres distillation): {attack_history['ber'][-1]:.2f}")
print(f"Accuracy Student: {attack_history['acc'][-1]:.1f}%")

if attack_history['ber'][-1] > 0.4:
    print("\n-> La marque Passport n'a PAS survecu a la distillation (BER eleve)")
else:
    print("\n-> La marque Passport a survecu a la distillation (BER faible)")